In [34]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install "transformers==4.31.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1" "evaluate" "sentencepiece"

ERROR: Could not find a version that satisfies the requirement sentencepiece--upgrade (from versions: none)
ERROR: No matching distribution found for sentencepiece--upgrade


In [1]:
!nvidia-smi

Tue Aug  8 07:04:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    47W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from datasets import load_dataset, Dataset
from random import randrange
from tqdm import tqdm

In [3]:
dt = load_dataset("lucasmccabe-lmi/CodeAlpaca-20k")

  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
dt["train"]

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 20022
})

In [5]:
dt["train"][0]

{'instruction': 'Create a function that takes a specific input and produces a specific output using any mathematical operators. Write corresponding code in Python.',
 'input': '',
 'output': 'def f(x):\n    """\n    Takes a specific input and produces a specific output using any mathematical operators\n    """\n    return x**2 + 3*x'}

In [6]:
dt = dt["train"].train_test_split(test_size=0.2, seed=7)

In [7]:
dt

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 16017
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 4005
    })
})

In [8]:
train_data = dt["train"]
valid_data = dt["test"]

In [9]:
def formatting_prompts_func(example):
    text = f"### Question: {example['instruction']}\n ### Answer: {example['output']}"
    return text

def chars_token_ratio(dataset, tokenizer, nb_examples=400):
    """
    Estimate the average number of characters per token in the dataset.
    """
    total_characters, total_tokens = 0, 0
    for _, example in tqdm(zip(range(nb_examples), iter(dataset)), total=nb_examples):
        text = formatting_prompts_func(example)
        total_characters += len(text)
        if tokenizer.is_fast:
            total_tokens += len(tokenizer(text).tokens())
        else:
            total_tokens += len(tokenizer.tokenize(text))

    return total_characters / total_tokens

In [10]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from trl.trainer import ConstantLengthDataset
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
import numpy as np


# import evaluate
# metric = evaluate.load("bleu")


In [12]:
model_id = "drive/MyDrive/flacon3b-int4-coding/merged_model/"

In [13]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
seq_length = 1024
chars_per_token = chars_token_ratio(train_data, tokenizer)

100%|██████████| 400/400 [00:00<00:00, 2087.50it/s]


In [14]:
train_dataset = ConstantLengthDataset(
        tokenizer,
        train_data,
        formatting_func=formatting_prompts_func,
        infinite=True,
        seq_length=seq_length,
        chars_per_token=chars_per_token,
    )

valid_dataset = ConstantLengthDataset(
    tokenizer,
    valid_data,
    formatting_func=formatting_prompts_func,
    infinite=False,
    seq_length=seq_length,
    chars_per_token=chars_per_token,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:246: UserWarning: The passed formatting_func has more than one argument. Usually that function should have a single argument `example` which corresponds to the dictonnary returned by each element of the dataset. Make sure you know what you are doing.
  warnings.warn(


In [15]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [16]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
base_model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, use_cache=False, device_map="auto")
base_model.config.pretraining_tp = 1
base_model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [25]:
# adjust r based on your budget and requirement, higher r if you can afford to train more params

peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=2,
        target_modules=["q_proj","v_proj"], #"k_proj" ,
        bias="none",
        task_type="CAUSAL_LM",
)

# optinally enable gradient checkpointing if model is huge > 13b for 48gb vram gpu
# or number of params to train are high
# model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(base_model)
model = get_peft_model(base_model, peft_config)

In [18]:
print_trainable_parameters(model)

trainable params: 1048576 || all params: 3501461504 || trainable%: 0.029946809319540645


In [19]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=2, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=2, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
              (v_

In [20]:
args = TrainingArguments(
    output_dir="drive/MyDrive/flacon3b-int4-coding",
    num_train_epochs=1,
    per_device_train_batch_size=6,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=2,
    save_strategy="steps",
    eval_steps=10,
    learning_rate=2e-4,
    bf16=True,
    tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
)

In [21]:
# def compute_metrics(eval_preds):
#     preds, labels = eval_preds
#     if isinstance(preds, tuple):
#         preds = preds[0]
#     decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
#     # Replace -100 in the labels as we can't decode them.
#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
#     result = {k: round(v * 100, 4) for k, v in result.items()}
#     prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
#     result["gen_len"] = np.mean(prediction_lens)
#     return result

In [22]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    # compute_metrics=compute_metrics,
    peft_config=peft_config,
    max_seq_length=seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=formatting_prompts_func,
    args=args,
)

In [27]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
2,1.425600
4,1.287400
6,1.182800


KeyboardInterrupt: ignored

In [24]:
# trainer.evaluate()
# you can evaluate if model is small and you have enough vram available

In [29]:
output_dir = args.output_dir
trainer.model.save_pretrained(output_dir)

In [30]:
# Free memory for merging weights
del base_model
torch.cuda.empty_cache()

In [31]:
from peft import AutoPeftModelForCausalLM
model = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map="auto", torch_dtype=torch.bfloat16)
model = model.merge_and_unload()

output_merged_dir = f"{output_dir}/merged_model/"
model.save_pretrained(output_merged_dir, safe_serialization=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]